### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
school_data_complete.head()
# Total Number of schools
unique_school_count = school_data_complete['school_name'].nunique()
unique_school_count
# Total Number of students
ind_student_count =  school_data_complete['student_name'].count()
ind_student_count
# Total Budget
ind_school_group = school_data_complete[['school_name','budget']]
ind_school_group = ind_school_group.drop_duplicates('school_name')
ind_school_group.head(15)
total_budget = ind_school_group['budget'].sum()
total_budget
# Average math score
avg_math_score = school_data_complete['math_score'].mean()
avg_math_score
# Average reading score
avg_read_score = school_data_complete['reading_score'].mean()
avg_read_score
# Pass math students
passing_math = school_data_complete[school_data_complete['math_score'] > 69]
passing_math_count = passing_math['math_score'].count()
passing_math_percent = (passing_math_count / ind_student_count) * 100
passing_math_percent
# Passing reading students
passing_read = school_data_complete[school_data_complete['reading_score'] >69]
passing_read_count = passing_read['reading_score'].count()
passing_read_percent = (passing_read_count / ind_student_count) * 100
passing_read_percent
# Percent Overall passing
overall_pass = school_data_complete[(school_data_complete['math_score'] > 69) & (school_data_complete['reading_score'] > 69)]
overall_pass_count = overall_pass['student_name'].count()
overall_pass_perent = (overall_pass_count / ind_student_count) * 100
overall_pass_perent
# Put it into a DataFrame
dist_sum = pd.DataFrame({'Total Schools': [unique_school_count],
                        'Total Students': [ind_student_count],
                        'Total Budget': [total_budget],
                        'Average Math Score': [avg_math_score],
                        'Average Reading Score': [avg_read_score],
                        '% Passing Math': [passing_math_percent],
                        '% Passing Reading': [passing_read_percent],
                        '% Overall Passing': [overall_pass_perent]})
# Format DataFrame to look better
dist_sum['Total Budget'] = dist_sum['Total Budget'].map('${:,.2f}'.format)
dist_sum['Total Students'] = dist_sum['Total Students'].map('{:,}'.format)
dist_sum['% Passing Math'] = dist_sum['% Passing Math'].map('{:,.2f}%'.format)
dist_sum['% Passing Reading'] = dist_sum['% Passing Reading'].map('{:,.2f}%'.format)
dist_sum['% Overall Passing'] = dist_sum['% Overall Passing'].map('{:,.2f}%'.format)
dist_sum

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
# Groupby school
school_group = school_data_complete.groupby('school_name')
# Total Students
# School Type 
school_type = school_data_complete.drop_duplicates('school_name')
# Set School Name as index
school_sum = school_type.set_index('school_name')
school_sum = school_sum.drop(['Student ID','student_name','gender','grade','School ID'],axis=1)
# Rename Columns
school_sum = school_sum.rename(columns={'school_name':'School Name',
                                       'type':'School Type',
                                       'size':'Total Students',
                                       'budget':'Total School Budget'})
school_sum
# Per Student Budget
school_sum['Per Student Budget'] = school_sum['Total School Budget'] / school_sum['Total Students']
# Average Math and Reading Score
avg_scores = school_group.mean()
avg_scores = avg_scores.drop(['Student ID','School ID','size','budget'],axis=1)
avg_scores = avg_scores.rename(columns={'school_name':'School Name',
                                       'reading_score':'Average Reading Score',
                                       'math_score':'Average Math Score'})
# Merge avg scores with school summary
school_sum = pd.merge(school_sum, avg_scores, on='school_name')
school_sum = school_sum.drop(['reading_score','math_score'],axis=1)
school_sum

# Percent Passing Math
school_group = school_data_complete.groupby(['school_name','Student ID'])
school_group = school_group.sum()
school_group = school_group[['math_score']]
school_group = school_group[school_group['math_score'] > 69]
school_group = school_group.groupby(['school_name'])
school_group = school_group.count()
school_group['% Passing Math'] = (school_group['math_score'] / school_sum['Total Students']) * 100
school_group = school_group.drop(columns={'math_score'})
# Merge back into school summary
school_sum = pd.merge(school_sum, school_group, on='school_name')
school_sum
# Percent Passing Reading
school_group = school_data_complete.groupby(['school_name','Student ID'])
school_group = school_group.sum()
school_group = school_group[['reading_score']]
school_group = school_group[school_group['reading_score'] > 69]
school_group = school_group.groupby(['school_name'])
school_group = school_group.count()
school_group['% Passing Reading'] = (school_group['reading_score'] / school_sum['Total Students']) * 100
school_group = school_group.drop(columns={'reading_score'})
# Merge back into school summary
school_sum = pd.merge(school_sum, school_group, on='school_name')
school_sum
# Percent Overall Passing
school_group = school_data_complete.groupby(['school_name','Student ID'])
school_group = school_group.sum()
school_group = school_group[['reading_score','math_score']]
school_group = school_group[(school_group['reading_score'] > 69) & (school_group['math_score'] > 69)]
school_group = school_group.groupby(['school_name'])
school_group = school_group.count()
school_group = school_group.drop(columns={'math_score'})
school_group['% Overall Passing'] = (school_group['reading_score'] / school_sum['Total Students']) * 100
school_group = school_group.drop(columns={'reading_score'})
# Merge back into school summary
school_sum = pd.merge(school_sum, school_group, on='school_name')
school_sum
# Format the DataTable
school_sum['Total School Budget'] = school_sum['Total School Budget'].map('${:,.2f}'.format)
school_sum['Per Student Budget'] = school_sum['Per Student Budget'].map('${:,.2f}'.format)
school_sum['Total Students'] = school_sum['Total Students'].map('{:,}'.format)
school_sum['% Passing Math'] = school_sum['% Passing Math'].map('{:,.2f}%'.format)
school_sum['% Passing Reading'] = school_sum['% Passing Reading'].map('{:,.2f}%'.format)
school_sum['% Overall Passing'] = school_sum['% Overall Passing'].map('{:,.2f}%'.format)
# Display table
school_sum 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,65.68%,81.32%,53.51%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,65.99%,80.74%,53.20%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.725724,83.359455,93.87%,95.85%,89.89%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.934412,77.289752,66.75%,80.86%,53.53%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.816757,83.351499,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.989488,83.274201,93.87%,96.54%,90.58%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,94.13%,97.04%,91.33%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,81.033963,77.048432,66.68%,81.93%,54.64%
Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,92.51%,96.25%,89.23%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
# Sort and display the top 5 performing schools (overall passing)
top_five = school_sum.sort_values(by='% Overall Passing', ascending=False)
top_five.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.975780,83.061895,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.848930,83.418349,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.816757,83.351499,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.989488,83.274201,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
# Display Bottom Performing Schools
bottom_five = school_sum.sort_values(by='% Overall Passing', ascending=True)
bottom_five.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,80.744686,76.842711,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,81.158020,76.711767,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635.00",$655.00,81.182722,76.629414,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,80.934412,77.289752,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,80.966394,77.072464,66.06%,81.22%,53.54%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [29]:
# Groupby school
school_group = school_data_complete.groupby('school_name')
# Create DataFrame of all 9th graders
grade_nine = school_data_complete.loc[school_data_complete['grade'] == '9th']
grade_nine = grade_nine.groupby('school_name')
grade_nine = grade_nine.mean()
grade_nine = grade_nine.drop(columns={'Student ID','School ID','size','budget'})
grade_nine_math = grade_nine.drop(columns={'reading_score'})
grade_nine_math = grade_nine_math.rename(columns={'math_score':'9th'})

# Create series of all 10th graders
grade_ten = school_data_complete[school_data_complete['grade'] == '10th']
grade_ten = grade_ten.groupby('school_name')
grade_ten = grade_ten.mean()
grade_ten = grade_ten.drop(columns={'Student ID','School ID','size','budget'})
grade_ten_math = grade_ten.drop(columns={'reading_score'})
grade_ten_math = grade_ten_math.rename(columns={'math_score':'10th'})

# Create DataFrame of all 11th graders
grade_elev = school_data_complete[school_data_complete['grade'] == '11th']
grade_elev = grade_elev.groupby('school_name')
grade_elev = grade_elev.mean()
grade_elev = grade_elev.drop(columns={'Student ID','School ID','size','budget'})
grade_elev_math = grade_elev.drop(columns={'reading_score'})
grade_elev_math = grade_elev_math.rename(columns={'math_score':'11th'})

# Create DataFrame of all 12th graders
grade_twlv = school_data_complete[school_data_complete['grade'] == '12th']
grade_twlv = grade_twlv.groupby('school_name')
grade_twlv = grade_twlv.mean()
grade_twlv = grade_twlv.drop(columns={'Student ID','School ID','size','budget'})
grade_twlv_math = grade_twlv.drop(columns={'reading_score'})
grade_twlv_math = grade_twlv_math.rename(columns={'math_score':'12th'})

# Combine them into one DataFrame
math_by_grade = pd.merge(grade_nine_math, grade_ten_math, on='school_name')
math_by_grade = pd.merge(math_by_grade, grade_elev_math, on='school_name')
math_by_grade = pd.merge(math_by_grade, grade_twlv_math, on='school_name')
math_by_grade = math_by_grade.rename(index={'school_name':'School Name'})
math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [30]:
# Use above Dataframes to compile the same for reading
# 9th grader reading scores by school
grade_nine_read = grade_nine.drop(columns={'math_score'})
grade_nine_read = grade_nine_read.rename(columns={'reading_score':'9th'})
# 10th grader reading scores by school
grade_ten_read = grade_ten.drop(columns={'math_score'})
grade_ten_read = grade_ten_read.rename(columns={'reading_score':'10th'})
# 9th grader reading scores by school
grade_elev_read = grade_elev.drop(columns={'math_score'})
grade_elev_read = grade_elev_read.rename(columns={'reading_score':'11th'})
# 9th grader reading scores by school
grade_twlv_read = grade_twlv.drop(columns={'math_score'})
grade_twlv_read = grade_twlv_read.rename(columns={'reading_score':'12th'})
# Combine them into one DataFrame
math_by_read = pd.merge(grade_nine_read, grade_ten_read, on='school_name')
math_by_read = pd.merge(math_by_read, grade_elev_read, on='school_name')
math_by_read = pd.merge(math_by_read, grade_twlv_read, on='school_name')
math_by_read = math_by_read.rename(index={'school_name':'School Name'})
math_by_read

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
